# Imports

In [ ]:
import os
import numpy as np
import subprocess
import pandas as pd
import requests
from urllib.parse import urljoin
import sqlite3
import json
import time
from datetime import datetime, timedelta
import re
from google.colab import drive
import random
from bs4 import BeautifulSoup
import csv
!pip install pydriller
from pydriller import Repository
from urllib.parse import urlparse

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 195.4/195.4 kB 4.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.0/66.0 kB 6.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 4.9 MB/s eta 0:00:00


In [ ]:
drive.mount('/content/gdrive/', force_remount=True)
# change location as per your convenience
# final_packages.txt (containing json dump of list of npm package names to be mined) should be present at this location
# database will be saved at this location
os.chdir("/content/gdrive/Shareddrives/ECS 260/final")

Mounted at /content/gdrive/


# Function Definitions

In [ ]:
def add_column_if_not_exists(cursor, table_name, column_name, column_definition):
    # check if the column already exists
    cursor.execute(f"PRAGMA table_info({table_name});")
    existing_columns = [column[1] for column in cursor.fetchall()]

    if column_name not in existing_columns:
        # add the column if it does not exist
        cursor.execute(f"ALTER TABLE {table_name} ADD COLUMN {column_name} {column_definition};")

In [ ]:
def GET(json_obj, keys, default = None):
  if not isinstance(keys, list):
    keys = [keys]
  current = json_obj
  try:
    for key in keys:
      if isinstance(current, list):
        key = int(key)
      current = current[key]
    return current
  except (TypeError, IndexError, KeyError):
    return default

def isValid(value):
  if isinstance(value, list):
    return not len(value) == 0
  return value != None and value != ""

def getN(arr):
  return len(arr) if isinstance(arr, list) else None

def isNumber(v):
  return isinstance(v, int)

In [ ]:
# fetch json from an API endpoint
def fetch_response_without_fail(url, params = {}, headers = {}, shouldScrap = False, verbose = True):
  retry_count = 0
  max_retries = 15
  time_interval = 10
  status = ""
  while retry_count < max_retries:
    try:
      response = requests.get(url, params=params, headers=headers)

      if response.status_code == 200:
        status = str(response.status_code)
        if shouldScrap:
          return (status, response)
        else:
          return (status, response.json())
      elif response.status_code == 404:
        status = str(response.status_code)
        if verbose:
          print(f"Error: {response.status_code}, skipping url {url}")
        return (status, None)
      elif response.status_code == 204:
        status = str(response.status_code)
        if verbose:
          print(f"Error: {response.status_code}, from url {url}")
        return (status, None)
      else:
        status = str(response.status_code)
        if verbose:
          print(f"Error: {response.status_code}")
          print(params)
        retry_count += 1
        if verbose:
          print(f"Retrying in {time_interval} seconds... (Retry {retry_count}/{max_retries})")
        if response.status_code == 429:
          retry_after = response.headers.get('Retry-After')
          if retry_after:
            if verbose:
              print("Retrying after", retry_after + 5, "Seconds")
            time.sleep(int(retry_after) + 5)
          else:
            time.sleep(time_interval)
        else:
          time.sleep(time_interval)

    except Exception as e:
      status = f"Exception occurred: {e}"
      if verbose:
        print(status)
      retry_count += 1
      if verbose:
        print(f"Retrying in {time_interval} seconds... (Retry {retry_count}/{max_retries})")
      time.sleep(time_interval)

  if verbose:
    print(f"Max retries reached. Unable to fetch data from {url}.")
  return (status, None)

In [ ]:
def get_lagging_dependencies_count(dependencies):
  status = "erred"
  if not isValid(dependencies):
    return (status, None)

  lagging_dependencies = 0
  for dependency, specified_version in dependencies.items():
    if specified_version == 'latest':
      continue

    (status, dependency_data) = fetch_response_without_fail(f'https://registry.npmjs.org/{dependency}/latest')
    if not status == '200':
      # skipping count if dependency doesn't exist for some reason
      continue
    latest_version = GET(dependency_data, 'version')
    if latest_version and specified_version != latest_version:
      lagging_dependencies += 1

  return ("success", lagging_dependencies)

In [ ]:
def get_github_url(repo_url):
  if not isValid(repo_url):
    return None
  repo_url_lowered = repo_url.lower()
  if not repo_url_lowered.find('gist.github.com') == -1:
    return None
  if not repo_url_lowered.find('gitee.com') == -1:
    return None

  github_idx = repo_url_lowered.find('github.com')
  if github_idx == -1:
    if repo_url.endswith('.git'):
      repo_url = repo_url[:-4]
      if ':' in repo_url:
        repo_url = repo_url.split(':')[-1]
      path_segments = repo_url.split('/')
      if len(path_segments) >= 2 and all(len(path) > 0 for path in path_segments[-2:]):
        author_repo = "/".join(path_segments[-2:])

        # checking for one special edge case
        if 'training/gitify' in author_repo:
          return None
        return f"https://github.com/{author_repo}"
      return None
    else:
      return None

  repo_url = repo_url[github_idx + len('github.com'):]
  path_segments = repo_url.split('/')
  if len(path_segments) >= 2 and all(len(path) > 0 for path in path_segments[-2:]):
    author_repo = "/".join(path_segments[-2:])
    author_repo = author_repo[:-4] if author_repo.endswith(".git") else author_repo
    author_repo = author_repo[:author_repo.find('.git#')] if '.git#' in author_repo else author_repo

    # checking for one special edge case
    if 'salesforce-experience-platform-emu/luvio-next' in author_repo:
      return None
    return f"https://github.com/{author_repo}"
  else:
    return None

def extract_github_url(repo_url):
  if not isValid(repo_url):
    return None
  repo_url_lowered = repo_url.lower()
  if not repo_url_lowered.find('gist.github.com') == -1:
    return None
  if not repo_url_lowered.find('gitee.com') == -1:
    return None

  github_idx = repo_url_lowered.find('github.com')
  formed_url = get_github_url(repo_url)
  if not repo_url_lowered.find('bitbucket') == -1 and github_idx == -1 and not formed_url == None:
    # trying to fetch the formed url and see if it gives a valid repo
    # as some bitbucket repos have moved to github and on coversion into
    # github url gives a valid repository
    (url_status, url_response) = fetch_response_without_fail(formed_url, shouldScrap=True, verbose=False)
    if url_status == '200':
      return formed_url
    return None

  return formed_url

In [ ]:
extract_github_url('git@bitbucket.org:Saartje87/blocks-v2.git')

'https://github.com/Saartje87/blocks-v2'

In [ ]:
extract_github_url('git+ssh://git@github.com/joseprl89/BitbucketPRAnalysis.git#id')

'https://github.com/joseprl89/BitbucketPRAnalysis'

In [ ]:
def extract_snyk_score(soup, category):
    selector = f'.scores li span:-soup-contains("{category}")'
    category_element = soup.select_one(selector)

    if category_element:
        category_score = category_element.find_next('span', class_='vue--pill__body').get_text(strip=True)
        return category_score.upper()
    else:
        return None

def scrap_snyk(package):
  status = "erred"
  try:
    status, response = fetch_response_without_fail(f"https://snyk.io/advisor/npm-package/{package}", shouldScrap=True)
    if not status == '200':
      return (status, None)
    response.raise_for_status() # Raise an HTTPError for bad responses
    soup = BeautifulSoup(response.text, 'html.parser')
    health_score_div = soup.select_one('.number span')
    if not health_score_div:
      if soup.find('div', class_='package-extra').find('p', class_='heading', string='This is a malicious package'):
        # For malicious packages, health score is 0
        health_score = 0
      else:
        # some error
        status = f"erred, element not found, status:{status}"
        health_score = None
      return (status, {
        "health_score": health_score,
        "security": None,
        "popularity": None,
        "maintenance": None,
        "community": None
      })
    health_score = health_score_div.get_text(strip=True).split('/')[0].strip()
    try:
      health_score = int(health_score)
    except:
      health_score

    return (status, {
      "health_score": health_score,
      "security": extract_snyk_score(soup, 'security'),
      "popularity": extract_snyk_score(soup, 'popularity'),
      "maintenance": extract_snyk_score(soup, 'maintenance'),
      "community": extract_snyk_score(soup, 'community')
    })
  except requests.exceptions.RequestException as e:
    status = str(response.status_code)
    print(f"Error scraping snyk for {package}: {e}")

  print(f"Error scraping snyk for {package}")
  return (status, None)

In [ ]:
def scrap_repo(repo_url):
  status = "erred"
  try:
    status, response = fetch_response_without_fail(repo_url, shouldScrap=True)
    if not status == '200':
      return (status, None)
    response.raise_for_status() # Raise an HTTPError for bad responses
    soup = BeautifulSoup(response.text, 'html.parser')
    border_grid_rows = soup.find_all(class_='BorderGrid-row')
    contributors = None
    for border_grid_row in border_grid_rows:
      text = border_grid_row.text.strip()
      if text.startswith('Contributors'):
        span_element = border_grid_row.find('span')
        if span_element:
          span_text = span_element.get_text(strip=True).replace(",", "")
          try:
            contributors = int(span_text)
          except:
            contributors = span_text

    commit_anchor_tag = soup.find("a", class_="react-last-commit-history-group")
    no_of_commits = None
    if commit_anchor_tag:
      # Find all span elements within the anchor tag
      spans = commit_anchor_tag.find_all("span")
      spans = spans[0].find_all("span")

      if len(spans) > 1:
        commit_count_span = spans[1]
        commit_count_text = commit_count_span.get_text(strip=True)
        no_of_commits = int(''.join(filter(str.isdigit, commit_count_text)))

    return (status, {"contributors": contributors, "no_of_commits": no_of_commits})
  except requests.exceptions.RequestException as e:
    status = str(response.status_code)
    print(f"Error scraping github repo {repo_url}: {e}")

  print(f"Error scraping github repo {repo_url}")
  return (status, None)

In [ ]:
def scrap_repo_net(repo_url):
  status = "erred"
  try:
    status, response = fetch_response_without_fail(repo_url + '/network/dependents', shouldScrap=True)
    if not status == '200':
      return (status, None)
    response.raise_for_status() # Raise an HTTPError for bad responses
    soup = BeautifulSoup(response.text, 'html.parser')
    btn_links = soup.find_all('a', class_='btn-link')
    dependants_count = None
    dependant_repos_count = None
    for btn_link in btn_links:
      text_content = btn_link.get_text(strip=True)

      if "Repositories" in text_content:
          dependants_count = int(''.join(filter(str.isdigit, (text_content.split()[0]))))
      elif "Packages" in text_content:
          dependant_repos_count = int(''.join(filter(str.isdigit, (text_content.split()[0]))))

    return (status, {"dependants_count": dependants_count, "dependant_repos_count": dependant_repos_count})
  except requests.exceptions.RequestException as e:
    status = str(response.status_code)
    print(f"Error scraping github repo {repo_url}: {e}")

  print(f"Error scraping github repo {repo_url}")
  return (status, None)

In [ ]:
def count_lines_of_code(repo_url):
  total_lines_of_code = 0
  filtered_lines_of_code = 0
  file_set = set()
  try:
    # Iterate over commits in the repository
    for commit in Repository(repo_url).traverse_commits():
      # Iterate over modified files in each commit
      for modified_file in commit.modified_files:
        # Check if the file is a source code file
        if modified_file.filename.endswith(('.py', '.json', '.md', '.html', '.yaml', '.yml', '.xml','.sh', '.css','.scss','.ts','.jsx','.tsx')):
          filtered_lines_of_code = filtered_lines_of_code+ (modified_file.added_lines - modified_file.deleted_lines)

        lines_of_code = modified_file.added_lines - modified_file.deleted_lines
        file_set.add(modified_file.filename)
        total_lines_of_code += lines_of_code
  except Exception as e:
    print("Didn't Work", repo_url)
    return None, None, None

  return total_lines_of_code, filtered_lines_of_code, len(file_set)

In [ ]:
def fetch_package_data_from_db(db_path, package_name, table_name):
  try:
    curr_conn = sqlite3.connect(db_path)
    curr_cursor = curr_conn.cursor()

    curr_cursor.execute(f"SELECT * FROM {table_name} WHERE package=?", (package_name,))
    curr_row = curr_cursor.fetchone()

    curr_conn.close()

    if curr_row:
      column_names = [description[0] for description in curr_cursor.description]
      package_data = dict(zip(column_names, curr_row))
      return package_data
    else:
      return None
  except:
    return None

# Distribution

In [ ]:
gh_token = '<your_github_token>'
li_api_key = '<your_libraries.io_token'

with open('final_packages.txt', "r") as file:
  final_packages = file.read()
package_list = json.loads(final_packages)

print("Total rows present: ", len(package_list))

Total rows present:  30000


# DB Init

In [ ]:
# init database and cursor
conn = sqlite3.connect("final_database.db")
cursor = conn.cursor()

In [ ]:
df = pd.read_sql_query(f"SELECT * FROM package_data", conn)
df

,package,npm_api_status,latest_version,no_of_versions,keywords,no_of_users,has_readme,has_homepage,last_modified_timestamp,created_timestamp,...,security,popularity,maintenance,community,avg_monthly_download,days_since_last_modification,security_numeric,popularity_numeric,maintenance_numeric,community_numeric
0,@gerrico/react-components,200,0.1.27,27.0,None,NaN,1.0,1.0,2022-08-12T10:57:20.535Z,2022-08-07T04:17:08.603Z,...,SECURITY REVIEW NEEDED,LIMITED,INACTIVE,LIMITED,14.0,559.0,50.0,20.0,5.0,20.0
1,express-simple-app-generator,200,1.0.5,6.0,"[""generate"", ""express""]",NaN,1.0,0.0,2023-09-26T09:25:54.330Z,2023-09-20T09:08:30.306Z,...,NO KNOWN SECURITY ISSUES,LIMITED,SUSTAINABLE,LIMITED,6.0,149.0,100.0,20.0,50.0,20.0
2,generator-giraffe,200,1.5.11,143.0,"[""yeoman-generator""]",NaN,1.0,1.0,2022-06-18T06:05:39.609Z,2013-10-22T13:10:45.431Z,...,SECURITY REVIEW NEEDED,LIMITED,INACTIVE,LIMITED,357.0,614.0,50.0,20.0,5.0,20.0
3,outdated-client,200,1.2.1,21.0,None,NaN,1.0,0.0,2022-05-12T09:34:06.984Z,2019-07-08T02:37:37.234Z,...,SECURITY REVIEW NEEDED,LIMITED,INACTIVE,LIMITED,58.0,651.0,50.0,20.0,5.0,20.0
4,@semi-bot/semi-theme-shopify,200,0.2.2,1.0,"[""semi-theme"", ""scss""]",NaN,1.0,0.0,2022-08-10T01:57:06.698Z,2022-08-10T01:57:06.319Z,...,NO KNOWN SECURITY ISSUES,LIMITED,INACTIVE,LIMITED,3.0,561.0,100.0,20.0,5.0,20.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
29995,haribotify,200,1.0.0,4.0,"[""browserify"", ""html"", ""components"", ""browseri...",NaN,1.0,1.0,2022-06-18T19:22:16.094Z,2016-01-27T10:34:18.517Z,...,NO KNOWN SECURITY ISSUES,LIMITED,INACTIVE,LIMITED,23.0,614.0,100.0,20.0,5.0,20.0
29996,eslint-config-sharecar,200,2.0.0,5.0,"[""eslint"", ""eslintconfig"", ""config"", ""airbnb"",...",NaN,1.0,1.0,2022-06-17T20:06:55.433Z,2017-04-20T07:37:46.942Z,...,NO KNOWN SECURITY ISSUES,LIMITED,INACTIVE,LIMITED,20.0,615.0,100.0,20.0,5.0,20.0
29997,webpack-to-ardoq,200,0.2.2,3.0,None,NaN,1.0,1.0,2023-10-11T07:55:07.798Z,2020-02-13T13:44:59.576Z,...,SECURITY REVIEW NEEDED,SMALL,INACTIVE,LIMITED,800.0,134.0,50.0,10.0,5.0,20.0
29998,zywave-content-search,200,0.0.6,6.0,None,NaN,1.0,0.0,2022-05-25T18:01:53.677Z,2022-03-04T19:23:37.896Z,...,SECURITY REVIEW NEEDED,LIMITED,INACTIVE,LIMITED,11.0,638.0,50.0,20.0,5.0,20.0


# Testing for errors in snyk data and re-mining

Certain values were scrapped as '?' despite having valid values on webpage.Hence scraping those packages again.

In [ ]:
cursor.execute("SELECT package FROM package_data WHERE (health_score IS NOT NULL OR security IS NOT NULL OR popularity IS NOT NULL OR maintenance IS NOT NULL OR community IS NOT NULL) AND snyk_scrapping_status = '404'")
rows_to_rescrap = cursor.fetchall()
cursor.execute("SELECT package FROM package_data WHERE health_score = '0' AND (security IS NOT NULL OR popularity IS NOT NULL OR maintenance IS NOT NULL OR community IS NOT NULL)")
rows_to_rescrap += cursor.fetchall()
cursor.execute("SELECT package FROM package_data WHERE (security IS NULL AND popularity IS NOT NULL)")
rows_to_rescrap += cursor.fetchall()
cursor.execute("SELECT package FROM package_data WHERE (health_score = '?' OR security = '?' OR popularity = '?' OR maintenance = '?' OR community = '?') AND snyk_scrapping_status = '200'")
rows_to_rescrap += cursor.fetchall()
packages_to_rescrap = [row[0] for row in rows_to_rescrap]

print(f"Rows to re-scrap: {len(packages_to_rescrap)}/{len(package_list)}")

Rows to re-scrap: 1/30000


In [ ]:
# Re-mining snyk data
rows_re_scraped = 0
for package in packages_to_rescrap:
  final_data = {}

  snyk_scrapping_status, snyk_scrapping_res = scrap_snyk(package)
  final_data['snyk_scrapping_status'] = snyk_scrapping_status
  print(f'Now mining for \n\t{package}\n\thttps://snyk.io/advisor/npm-package/{package}')

  if snyk_scrapping_status == '200':
    final_data['health_score'] = snyk_scrapping_res['health_score']
    final_data['security'] = snyk_scrapping_res['security']
    final_data['popularity'] = snyk_scrapping_res['popularity']
    final_data['maintenance'] = snyk_scrapping_res['maintenance']
    final_data['community'] = snyk_scrapping_res['community']

    print("snyk.io API mined")
  else:
    final_data['health_score'] = final_data['security'] = final_data['popularity'] = final_data['maintenance'] = final_data['community'] = None
    print(f'Failed for {package}')

  if len(final_data) > 0:
    print(final_data)
    print()
    updates = [(column, value) for column, value in final_data.items()]

    # Execute the UPDATE statement
    update_statement = f"UPDATE package_data SET {', '.join([f'{col} = ?' for col, _ in updates])} WHERE package = ?"
    cursor.execute(update_statement, [val for _, val in updates] + [package])

    # Commit the changes and close the database connection
    conn.commit()

  rows_re_scraped += 1
  print("--------------------------------------------------------------------------------------------------------------------------------------------")
  print(f"Rows re-mined so far: {rows_re_scraped}/{len(packages_to_rescrap)}")
  print("--------------------------------------------------------------------------------------------------------------------------------------------")
  print()

Now mining for 
	kuda-delsia-p2p-binance-axxz5zf0v
	https://snyk.io/advisor/npm-package/kuda-delsia-p2p-binance-axxz5zf0v
snyk.io API mined
{'snyk_scrapping_status': '200', 'health_score': 20, 'security': '?', 'popularity': 'LIMITED', 'maintenance': 'INACTIVE', 'community': 'LIMITED'}

--------------------------------------------------------------------------------------------------------------------------------------------
Rows re-mined so far: 1/1
--------------------------------------------------------------------------------------------------------------------------------------------



# Testing for errors in github url extraction and re-mining

Certain urls were extracted as `None` due to error in `extract_github_url` function.

In [ ]:
cursor.execute("SELECT package, repository_url, git_repository_url, git_repository_status, git_repository_url_final FROM package_data")
rows = cursor.fetchall()

packages_to_update = []
rows_not_to_mine = []
old_rows = []
for row in rows:
  package, repository_url, git_repository_url, git_repository_url_status, git_repository_url_final = row

  # Calculate the new extracted url
  calculated_url = extract_github_url(repository_url)

  # Check if the calculated url is different from the extracted url
  if git_repository_url != calculated_url:
    packages_to_update.append((package, calculated_url, repository_url))
    old_rows.append((git_repository_url, repository_url))
    if not isValid(calculated_url):
      rows_not_to_mine.append(package)
  if not isValid(git_repository_url_final) and git_repository_url_status == '200':
    packages_to_update.append((package, calculated_url, repository_url))
    old_rows.append((git_repository_url, repository_url))
    if not isValid(calculated_url):
      rows_not_to_mine.append(package)
  if isValid(git_repository_url_final) and git_repository_url_status == '404':
    packages_to_update.append((package, calculated_url, repository_url))
    old_rows.append((git_repository_url, repository_url))
    if not isValid(calculated_url):
      rows_not_to_mine.append(package)

print(f"Rows to re-mine: {len(packages_to_update)}/{len(package_list)}")
print(f"Rows not to re-mine: {len(rows_not_to_mine)}/{len(package_list)}")

Rows to re-mine: 0/30000
Rows not to re-mine: 0/30000


In [ ]:
packages_to_update

[]

In [ ]:
# Re-mining github data
rows_done = 0
for package, git_repository_url, repository_url in packages_to_update:
  # mining github API data
  git_repository_url_final = None
  final_data = {'git_repository_url': git_repository_url, 'git_repository_url_final': git_repository_url_final, 'git_repository_status': None}
  continue_mining = isValid(git_repository_url)

  print(f"Now checking {git_repository_url}\n\tfor {package}\n\tfrom {repository_url}")
  if continue_mining:
    # getting redirected gh url
    try:
      git_repository_url_final = requests.get(git_repository_url, allow_redirects=False).headers['Location']
    except Exception as e:
      git_repository_url_final = git_repository_url

    # checking if url is valid
    try:
      res = requests.get(git_repository_url_final)
      git_repository_status = str(res.status_code)
    except Exception as e:
      git_repository_status = f'Exception: {e}'
    final_data['git_repository_status'] = git_repository_status
    continue_mining = git_repository_status == '200'

  # continue github mining
  if continue_mining:
    print("Continuing...", git_repository_url_final)
    final_data['git_repository_url_final'] = git_repository_url_final

    headers = {
      'Authorization': f'token {gh_token}',
      'Accept': 'application/vnd.github.v3+json'
    }

    repo_owner, repo_name = git_repository_url_final.split('/')[-2:]
    gh_api_url = f"https://api.github.com/repos/{repo_owner}/{repo_name}"

    # mining github API data
    gh_api_status, gh_api_res = fetch_response_without_fail(gh_api_url, headers=headers)
    final_data['gh_api_status'] = gh_api_status
    if gh_api_status == '200':
      forks = GET(gh_api_res, 'forks_count')
      final_data['forks'] = forks

      stars = GET(gh_api_res, 'stargazers_count')
      final_data['stars'] = stars

      watchers = GET(gh_api_res, 'subscribers_count')
      final_data['watchers'] = watchers

      start_date_timestamp = GET(gh_api_res, 'created_at')
      if isValid(start_date_timestamp):
        start_date = datetime.strptime(start_date_timestamp, "%Y-%m-%dT%H:%M:%SZ").date()
        # age till Feb 23, 2024
        end_date = datetime(2024, 2, 23).date()
        total_days = (end_date - start_date).days
      else:
        total_days = None


    # mining github issues API data
    gh_issues_api_url = f"https://api.github.com/search/issues?q=is:issue%20is:open%20repo:{repo_owner}/{repo_name}"
    gh_issue_api_status, gh_issues_api_res = fetch_response_without_fail(gh_issues_api_url, headers={'Authorization': f'token {gh_token}'})
    final_data['gh_issue_api_status'] = gh_issue_api_status
    if gh_issue_api_status == '200':
      issues = GET(gh_issues_api_res, 'total_count')
      final_data['issues'] = issues


    # mining github pr API data
    gh_pr_api_url = f"https://api.github.com/search/issues?q=is:pr%20is:open%20repo:{repo_owner}/{repo_name}"
    gh_pr_api_status, gh_pr_api_res = fetch_response_without_fail(gh_pr_api_url, headers={'Authorization': f'token {gh_token}'})
    final_data['gh_pr_api_status'] = gh_pr_api_status
    if gh_pr_api_status == '200':
      pr = GET(gh_pr_api_res, 'total_count')
      final_data['pr'] = pr


    # scrapping repository data
    gh_scrapping_status, gh_scrapping_res = scrap_repo(git_repository_url_final)
    final_data['gh_scrapping_status'] = gh_scrapping_status

    if gh_scrapping_status == '200':
      final_data['contributors'] = gh_scrapping_res['contributors']
      final_data['no_of_commits'] = gh_scrapping_res['no_of_commits']
      final_data['avg_commit_freq'] = gh_scrapping_res['no_of_commits'] / total_days if isValid(total_days) and isValid(gh_scrapping_res['no_of_commits']) else None


    # scrapping repository network data
    gh_net_scrapping_status, gh_net_scrapping_res = scrap_repo_net(git_repository_url_final)
    final_data['gh_net_scrapping_status'] = gh_net_scrapping_status

    if gh_net_scrapping_status == '200':
      final_data['dependants_count'] = gh_net_scrapping_res['dependants_count']
      final_data['dependant_repos_count'] = gh_net_scrapping_res['dependant_repos_count']

    # pydriller
    try:
      old_data = fetch_package_data_from_db('FINAL_PACKAGE_DATA.db', package, 'final_package_data')
      if isValid(old_data) and (isValid(old_data['Lines Of Codes']) or isValid(old_data['Number of Files']) or isValid(old_data['Filtered Lines of Codes'])):
        final_data['total_lines_of_code'] = old_data['Lines Of Codes']
        final_data['filtered_lines_of_code'] = old_data['Number of Files']
        final_data['no_of_files'] = old_data['Filtered Lines of Codes']
      else:
        # some repo urls have been same for multiple packages so fetching data from them instead of recalculating
        cursor.execute("SELECT total_lines_of_code, filtered_lines_of_code, no_of_files FROM package_data WHERE git_repository_url_final = ? AND (total_lines_of_code IS NOT NULL OR filtered_lines_of_code IS NOT NULL OR no_of_files IS NOT NULL)", [git_repository_url_final])
        result_to_check = cursor.fetchone()
        if isValid(result_to_check):
          final_data['total_lines_of_code'], final_data['filtered_lines_of_code'], final_data['no_of_files'] = result_to_check
        else:
          # If the result is empty or values are None, calculate the lines of code
          total_lines_of_code, filtered_lines_of_code, no_of_files = count_lines_of_code(git_repository_url_final)
          final_data['total_lines_of_code'] = total_lines_of_code
          final_data['filtered_lines_of_code'] = filtered_lines_of_code
          final_data['no_of_files'] = no_of_files

    except Exception as e:
      print(f"pydriller failed for {package}\n{e}")
  else:
    print("Skipping...", git_repository_url_final)

  if len(final_data) > 0:
    updates = [(column, value) for column, value in final_data.items()]

    # Execute the UPDATE statement
    update_statement = f"UPDATE package_data SET {', '.join([f'{col} = ?' for col, _ in updates])} WHERE package = ?"
    cursor.execute(update_statement, [val for _, val in updates] + [package])

    # Commit the changes and close the database connection
    conn.commit()

  rows_done += 1
  print("--------------------------------------------------------------------------------------------------------------------------------------------")
  print(f"Rows done so far: {rows_done}/{len(packages_to_update)}")
  print("--------------------------------------------------------------------------------------------------------------------------------------------")
  print()

In [ ]:
conn.close()

# Testing for errors in contributors count and re-mining

Certain urls were contributors count were extracted as `None` despite having valid github urls as they dont have contributors section on their repository page. Hence fetching the api to get contributors count, assuming its less than 30.

In [ ]:
cursor.execute("SELECT package, git_repository_url_final, gh_scrapping_status FROM package_data WHERE git_repository_url_final IS NOT NULL AND contributors IS NULL")
rows_to_remine = cursor.fetchall()

packages_to_remine = [(package, git_repository_url_final) for (package, git_repository_url_final, gh_scrapping_status) in rows_to_remine]

print(f"Rows to re-mine: {len(packages_to_remine)}/{len(package_list)}")

Rows to re-mine: 45/30000


In [ ]:
# Re-mining github data
rows_re_mined = 0
for package, git_repository_url_final in packages_to_remine:
  final_data = {}

  headers = {
    'Authorization': f'token {gh_token}',
    'Accept': 'application/vnd.github.v3+json'
  }

  repo_owner, repo_name = git_repository_url_final.split('/')[-2:]
  gh_contributors_api_url = f"https://api.github.com/repos/{repo_owner}/{repo_name}/contributors"
  print(f'Now mining for \n\t{package}\n\t{git_repository_url_final}\n\t{gh_contributors_api_url}')

  gh_contributors_api_status, gh_contributors_api_res = fetch_response_without_fail(gh_contributors_api_url, headers=headers)
  if gh_contributors_api_status == '200':
    contributors = getN(gh_contributors_api_res)
    if isValid(contributors) and contributors < 30:
      final_data['contributors'] = contributors
      print('Contributors mined! -->', contributors)
    else:
      # fetching contributors recursively through api
      curr_contributors_count = 0
      curr_page = 1
      params = {'page': curr_page}
      while contributors != 0:
        curr_contributors_count += contributors
        curr_page += 1
        params = {'page': curr_page}
        gh_contributors_api_status, gh_contributors_api_res = fetch_response_without_fail(gh_contributors_api_url, params=params, headers=headers)
        contributors = getN(gh_contributors_api_res)
      final_data['contributors'] = curr_contributors_count
      print('Recursively contributors mined! -->', curr_contributors_count)
  elif gh_contributors_api_status == '204':
    scrap_status, scrap_res = fetch_response_without_fail(f'{git_repository_url_final}/graphs/contributors', shouldScrap=True)
    if scrap_status == '200':
      try:
        soup = BeautifulSoup(scrap_res, 'html.parser')
        contrib_person_elements = soup.find_all('li', class_='contrib-person')
        final_data['contributors'] = len(contrib_person_elements)
      except:
        print('Failed further scraping')
    else:
      print(f'Failed for \n\t{git_repository_url_final}/graphs/contributors')
  else:
    print(f'Failed for \n\t{gh_contributors_api_url}\n\t{git_repository_url_final}')

  if len(final_data) > 0:
    updates = [(column, value) for column, value in final_data.items()]

    # Execute the UPDATE statement
    update_statement = f"UPDATE package_data SET {', '.join([f'{col} = ?' for col, _ in updates])} WHERE package = ?"
    cursor.execute(update_statement, [val for _, val in updates] + [package])

    # Commit the changes and close the database connection
    conn.commit()

  rows_re_mined += 1
  print("--------------------------------------------------------------------------------------------------------------------------------------------")
  print(f"Rows re-mined so far: {rows_re_mined}/{len(packages_to_remine)}")
  print("--------------------------------------------------------------------------------------------------------------------------------------------")
  print()

Now mining for 
	pratik-npm-package
	https://github.com/PrimeOfC-star/pratik-npm-package
	https://api.github.com/repos/PrimeOfC-star/pratik-npm-package/contributors
Error: 204, from url https://api.github.com/repos/PrimeOfC-star/pratik-npm-package/contributors
Failed further scraping
--------------------------------------------------------------------------------------------------------------------------------------------
Rows re-mined so far: 1/45
--------------------------------------------------------------------------------------------------------------------------------------------

Now mining for 
	formvader
	https://github.com/tony5hot/formvader
	https://api.github.com/repos/tony5hot/formvader/contributors
Error: 204, from url https://api.github.com/repos/tony5hot/formvader/contributors
Failed further scraping
--------------------------------------------------------------------------------------------------------------------------------------------
Rows re-mined so far: 2/45
---

# Testing for errors in github issue and pr extraction and re-mining

Certain github repo stats (eg. `issue`, `pr`, etc.) were not fetched, re-mining the same.

In [ ]:
cursor.execute("SELECT package, git_repository_url, git_repository_url_final, repository_url FROM package_data WHERE git_repository_url_final IS NOT NULL AND (issues IS NULL OR pr IS NULL OR stars IS NULL OR forks IS NULL OR watchers IS NULL)")
packages_to_update = cursor.fetchall()
cursor.execute("SELECT package, git_repository_url, git_repository_url_final, repository_url FROM package_data WHERE git_repository_url_final IS NULL AND (issues IS NOT NULL OR pr IS NOT NULL OR stars IS NOT NULL OR forks IS NOT NULL OR watchers IS NOT NULL)")
packages_to_update += cursor.fetchall()

print(f"Rows to re-mine: {len(packages_to_update)}/{len(package_list)}")

Rows to re-mine: 0/30000


In [ ]:
# Re-mining github data
rows_done = 0
for package, git_repository_url_final in packages_to_update:
  # mining github API data
  final_data = {}

  print(f"Now checking {git_repository_url_final}\n\tfor {package}")

  headers = {
    'Authorization': f'token {gh_token}',
    'Accept': 'application/vnd.github.v3+json'
  }

  repo_owner, repo_name = git_repository_url_final.split('/')[-2:]
  gh_api_url = f"https://api.github.com/repos/{repo_owner}/{repo_name}"
  print(f"\tforks, stars, watchers: {gh_api_url}")

  # mining github API data
  gh_api_status, gh_api_res = fetch_response_without_fail(gh_api_url, headers=headers)
  final_data['gh_api_status'] = gh_api_status
  total_days = None
  if gh_api_status == '200':
    forks = GET(gh_api_res, 'forks_count')
    final_data['forks'] = forks

    stars = GET(gh_api_res, 'stargazers_count')
    final_data['stars'] = stars

    watchers = GET(gh_api_res, 'subscribers_count')
    final_data['watchers'] = watchers

    start_date_timestamp = GET(gh_api_res, 'created_at')
    if isValid(start_date_timestamp):
      start_date = datetime.strptime(start_date_timestamp, "%Y-%m-%dT%H:%M:%SZ").date()
      # age till Feb 23, 2024
      end_date = datetime(2024, 2, 23).date()
      total_days = (end_date - start_date).days


  # mining github issues API data
  gh_issues_api_url = f"https://api.github.com/search/issues?q=is:issue%20is:open%20repo:{repo_owner}/{repo_name}"
  print(f"\tissues: {gh_issues_api_url}")
  gh_issue_api_status, gh_issues_api_res = fetch_response_without_fail(gh_issues_api_url, headers={'Authorization': f'token {gh_token}'})
  final_data['gh_issue_api_status'] = gh_issue_api_status
  if gh_issue_api_status == '200':
    issues = GET(gh_issues_api_res, 'total_count')
    final_data['issues'] = issues


  # mining github pr API data
  gh_pr_api_url = f"https://api.github.com/search/issues?q=is:pr%20is:open%20repo:{repo_owner}/{repo_name}"
  print(f"\tpr: {gh_pr_api_url}")
  gh_pr_api_status, gh_pr_api_res = fetch_response_without_fail(gh_pr_api_url, headers={'Authorization': f'token {gh_token}'})
  final_data['gh_pr_api_status'] = gh_pr_api_status
  if gh_pr_api_status == '200':
    pr = GET(gh_pr_api_res, 'total_count')
    final_data['pr'] = pr


  # scrapping repository data
  gh_scrapping_status, gh_scrapping_res = scrap_repo(git_repository_url_final)
  final_data['gh_scrapping_status'] = gh_scrapping_status

  if gh_scrapping_status == '200':
    final_data['contributors'] = gh_scrapping_res['contributors']
    final_data['no_of_commits'] = gh_scrapping_res['no_of_commits']
    final_data['avg_commit_freq'] = gh_scrapping_res['no_of_commits'] / total_days if isValid(total_days) and isValid(gh_scrapping_res['no_of_commits']) else None

  # scrapping repository network data
  gh_net_scrapping_status, gh_net_scrapping_res = scrap_repo_net(git_repository_url_final)
  final_data['gh_net_scrapping_status'] = gh_net_scrapping_status

  if gh_net_scrapping_status == '200':
    final_data['dependants_count'] = gh_net_scrapping_res['dependants_count']
    final_data['dependant_repos_count'] = gh_net_scrapping_res['dependant_repos_count']

  if len(final_data) > 0:
    # updates = [(column, value) for column, value in final_data.items()]

    # # Execute the UPDATE statement
    # update_statement = f"UPDATE package_data SET {', '.join([f'{col} = ?' for col, _ in updates])} WHERE package = ?"
    # cursor.execute(update_statement, [val for _, val in updates] + [package])

    # # Commit the changes and close the database connection
    # conn.commit()
    print(final_data)

  rows_done += 1
  print("--------------------------------------------------------------------------------------------------------------------------------------------")
  print(f"Rows done so far: {rows_done}/{len(packages_to_update)}")
  print("--------------------------------------------------------------------------------------------------------------------------------------------")
  print()

In [ ]:
conn.close()

# Testing for errors in py-drilling

Certain rows were skipped due to error in function `fetch_package_data_from_db` to fetch data from old database

In [ ]:
cursor.execute("SELECT package, git_repository_url_final FROM package_data WHERE git_repository_url_final IS NOT NULL AND (total_lines_of_code IS NULL OR filtered_lines_of_code IS NULL OR no_of_files IS NULL)")
rows_to_drill = cursor.fetchall()
cursor.execute("SELECT package, git_repository_url_final FROM package_data WHERE git_repository_url_final IS NULL AND (total_lines_of_code IS NOT NULL OR filtered_lines_of_code IS NOT NULL OR no_of_files IS NOT NULL)")
rows_to_drill += cursor.fetchall()

packages_to_drill = [(package, git_repository_url_final) for (package, git_repository_url_final) in rows_to_drill]

print(f"Rows to drill: {len(packages_to_drill)}/{len(package_list)}")

Rows to drill: 0/30000


In [ ]:
# Re-mining github data
rows_drilled = 0
for package, git_repository_url_final in packages_to_drill:
  final_data = {'total_lines_of_code': None, 'filtered_lines_of_code': None, 'no_of_files': None}

  # pydrilling
  print(f"Now drilling {git_repository_url_final}\n\tfor {package}")
  try:
    old_data = fetch_package_data_from_db('FINAL_PACKAGE_DATA.db', package, 'final_package_data')
    if isValid(old_data) and (isValid(old_data['Lines Of Codes']) or isValid(old_data['Number of Files']) or isValid(old_data['Filtered Lines of Codes'])):
      final_data['total_lines_of_code'] = old_data['Lines Of Codes']
      final_data['filtered_lines_of_code'] = old_data['Number of Files']
      final_data['no_of_files'] = old_data['Filtered Lines of Codes']
    else:
      # some repo urls have been same for multiple packages so fetching data from them instead of recalculating
      cursor.execute("SELECT total_lines_of_code, filtered_lines_of_code, no_of_files FROM package_data WHERE git_repository_url_final = ? AND (total_lines_of_code IS NOT NULL OR filtered_lines_of_code IS NOT NULL OR no_of_files IS NOT NULL)", [git_repository_url_final])
      result_to_check = cursor.fetchone()
      if isValid(result_to_check):
        final_data['total_lines_of_code'], final_data['filtered_lines_of_code'], final_data['no_of_files'] = result_to_check
      else:
        # If the result is empty or values are None, calculate the lines of code
        total_lines_of_code, filtered_lines_of_code, no_of_files = count_lines_of_code(git_repository_url_final)
        final_data['total_lines_of_code'] = total_lines_of_code
        final_data['filtered_lines_of_code'] = filtered_lines_of_code
        final_data['no_of_files'] = no_of_files

  except Exception as e:
    print(f"pydriller failed for {package}\n{e}")

  if len(final_data) > 0:
    updates = [(column, value) for column, value in final_data.items()]

    # Execute the UPDATE statement
    update_statement = f"UPDATE package_data SET {', '.join([f'{col} = ?' for col, _ in updates])} WHERE package = ?"
    cursor.execute(update_statement, [val for _, val in updates] + [package])

    # Commit the changes and close the database connection
    conn.commit()

  rows_drilled += 1
  print("--------------------------------------------------------------------------------------------------------------------------------------------")
  print(f"Rows drilled so far: {rows_drilled}/{len(packages_to_drill)}")
  print("--------------------------------------------------------------------------------------------------------------------------------------------")
  print()

In [ ]:
# removing invalid values
cursor.execute("UPDATE package_data SET sloc = NULL WHERE sloc = -1 OR sloc = '-1'")
conn.commit()
cursor.execute("UPDATE package_data SET total_lines_of_code = NULL WHERE total_lines_of_code = -1 OR total_lines_of_code = '-1'")
conn.commit()
cursor.execute("UPDATE package_data SET no_of_files = NULL WHERE no_of_files = -1 OR no_of_files = '-1'")
conn.commit()

In [ ]:
conn.close()

# Renaming some columns

In [ ]:
cursor.execute(f"ALTER TABLE package_data RENAME COLUMN filtered_lines_of_code TO sloc")
conn.commit()

# Adding a new metric derived from `last_modified_timestamp` in days

In [ ]:
df['last_modified_timestamp'] = pd.to_datetime(df['last_modified_timestamp'])

# comparing to 23 Feb, 2023
comparison_date = pd.to_datetime('2024-02-23')

df['last_modified_timestamp'] = df['last_modified_timestamp'].dt.tz_localize(None)
df['days_since_last_modification'] = (comparison_date - df['last_modified_timestamp']).dt.days

# alter_query = "ALTER TABLE package_data ADD COLUMN days_since_last_modification INTEGER;"
# conn.execute(alter_query)

update_query = """
    UPDATE package_data
    SET days_since_last_modification = ?
    WHERE package = ?
"""

for index, row in df.iterrows():
  values = (row['days_since_last_modification'], row['package'])
  # print("Updating package:", row['package'])
  conn.execute(update_query, values)

conn.commit()
conn.close()

# Converting text values into numerical values

In [ ]:
# cursor.execute("ALTER TABLE package_data ADD COLUMN security_numeric INTEGER")
# conn.commit()

security_update_query = """
UPDATE package_data
SET security_numeric =
    CASE
        WHEN security = 'SECURITY REVIEW NEEDED' THEN 50
        WHEN security = 'NO KNOWN SECURITY ISSUES' THEN 100
        WHEN security = '?' THEN NULL
        WHEN security = 'SECURITY ISSUES FOUND' THEN 10
        ELSE NULL
    END
"""
cursor.execute(security_update_query)
conn.commit()

In [ ]:
# cursor.execute("ALTER TABLE package_data ADD COLUMN popularity_numeric INTEGER")
# conn.commit()

popularity_update_query = """
UPDATE package_data
SET popularity_numeric =
    CASE
        WHEN popularity = 'LIMITED' THEN 20
        WHEN popularity = 'INFLUENTIAL PROJECT' THEN 100
        WHEN popularity = 'SMALL' THEN 10
        WHEN popularity = 'RECOGNIZED' THEN 60
        WHEN popularity = 'POPULAR' THEN 50
        WHEN popularity = 'KEY ECOSYSTEM PROJECT' THEN 80
        ELSE NULL
    END
"""
cursor.execute(popularity_update_query)
conn.commit()

In [ ]:
# cursor.execute("ALTER TABLE package_data ADD COLUMN maintenance_numeric INTEGER")
# conn.commit()

maintenance_update_query = """
UPDATE package_data
SET maintenance_numeric =
    CASE
        WHEN maintenance = 'INACTIVE' THEN 5
        WHEN maintenance = 'SUSTAINABLE' THEN 50
        WHEN maintenance = 'HEALTHY' THEN 100
        ELSE NULL
    END
"""
cursor.execute(maintenance_update_query)
conn.commit()

In [ ]:
# cursor.execute("ALTER TABLE package_data ADD COLUMN community_numeric INTEGER")
# conn.commit()

community_update_query = """
UPDATE package_data
SET community_numeric =
    CASE
        WHEN community = 'LIMITED' THEN 20
        WHEN community = 'SUSTAINABLE' THEN 50
        WHEN community = 'ACTIVE' THEN 100
        ELSE NULL
    END
"""
cursor.execute(community_update_query)
conn.commit()

In [ ]:
# accounting for cases when health score is zero
maintenance_update_query = """
UPDATE package_data
SET security_numeric = 0, popularity_numeric = 0, community_numeric = 0, maintenance_numeric = 0 WHERE health_score = 0
"""
cursor.execute(maintenance_update_query)
conn.commit()